In [1]:
import pandas as pd

covid_df = pd.read_csv('../data/covidtesting.csv', engine='python')
covid_age_df = pd.read_csv('../data/percent_positive_by_agegrp.csv', engine='python')
vaccine_df = pd.read_csv('../data/vaccine_doses.csv', engine='python')

In [2]:
covid_df.head()

,Reported Date,Confirmed Negative,Presumptive Negative,Presumptive Positive,Confirmed Positive,Resolved,Deaths,Total Cases,Total patients approved for testing as of Reporting Date,Total tests completed in the last day,...,Number of patients in ICU on a ventilator due to COVID-19,Num. of patients in ICU on a ventilator testing positive,Num. of patients in ICU on a ventilator testing negative,Total Positive LTC Resident Cases,Total Positive LTC HCW Cases,Total LTC Resident Deaths,Total LTC HCW Deaths,Total_Lineage_B.1.1.7,Total_Lineage_B.1.351,Total_Lineage_P.1
0,2020-01-26,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-27,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-28,NaN,NaN,1.0,1.0,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-30,NaN,NaN,0.0,2.0,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-31,NaN,NaN,0.0,2.0,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
covid_age_df.head()

,DATE,age_category,percent_positive_7d_avg
0,2020-05-01,0to13,0.0340
1,2020-05-02,0to13,0.0350
2,2020-05-03,0to13,0.0360
3,2020-05-04,0to13,0.0359
4,2020-05-05,0to13,0.0355


In [4]:
vaccine_df.head()

,report_date,previous_day_doses_administered,total_doses_administered,total_doses_in_fully_vaccinated_individuals,total_individuals_fully_vaccinated
0,2020-12-24,NaN,"10,756",NaN,NaN
1,2020-12-30,"4,595","18,603",NaN,NaN
2,2020-12-31,"5,463","23,502",NaN,NaN
3,2021-01-01,"5,415","28,887",NaN,NaN
4,2021-01-02,"4,305","33,191",NaN,NaN
